### Translation Task:

La traduction est une tâche de type séquence-à-séquence, similaire au résumé de texte. Elle peut également être adaptée à d'autres problèmes de ce genre, comme le transfert de style (par exemple, traduire un texte formel en un texte plus décontracté) ou la génération de réponses à des questions en fonction d'un contexte.

Si vous disposez d'un grand corpus de textes dans deux langues ou plus, vous pouvez entraîner un nouveau modèle de traduction à partir de zéro. Toutefois, il est souvent plus rapide de faire un fine-tuning d’un modèle de traduction existant, comme un modèle multilingue tel que mT5 ou mBART, ou un modèle spécialisé pour la traduction d’une langue à une autre.

Dans cette section, un modèle Marian pré-entraîné pour la traduction de l’anglais vers le français sera ajusté (fine-tuned) en utilisant le jeu de données KDE4. Ce modèle a été initialement entraîné sur un large corpus de textes en anglais et en français, et nous allons améliorer ses performances après l’étape de fine-tuning.

Une fois l’entraînement terminé, le modèle pourra faire des prédictions de traduction.

#### 1- Preparing the data


In [1]:
# Import librairies
import transformers
from datasets import load_dataset
from transformers import pipeline
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM

In [2]:
# Lire our dataset
dataset = load_dataset("kde4", lang1 = "en", lang2 = "fr")
dataset

c:\Users\Zakaria-Laptop\AppData\Local\Programs\Python\Python312\Lib\site-packages\datasets\load.py:1454: FutureWarning: The repository for kde4 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/kde4
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 210173
    })
})

Nous avons 210 173 paires de phrases, mais elles sont regroupées en un seul ensemble, ce qui signifie que nous devons créer notre propre ensemble de validation. Un objet Dataset possède une méthode train_test_split() qui peut nous aider. Nous allons fournir une graine (seed) pour garantir la reproductibilité.

In [3]:
# Splitter our data in train and test
split_datasets = dataset["train"].train_test_split(train_size=0.9, seed=20)

#Rename our test to "validation"
split_datasets["validation"] = split_datasets.pop('test')

split_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 189155
    })
    validation: Dataset({
        features: ['id', 'translation'],
        num_rows: 21018
    })
})

In [4]:
split_datasets["train"][10]["translation"]

{'en': 'Text Cursor Movement', 'fr': 'Mouvements du curseur de texte'}

Nous obtenons un dictionnaire contenant deux phrases dans les langues demandées. Une particularité de ce jeu de données, qui contient beaucoup de termes techniques en informatique, est que tous ces termes sont entièrement traduits en français. Cependant, les ingénieurs français laissent souvent les mots spécifiques à l'informatique en anglais lorsqu'ils parlent. Par exemple, le mot « threads » pourrait apparaître tel quel dans une phrase française, surtout dans une conversation technique, mais dans ce jeu de données, il est traduit par l'expression plus correcte « fils de discussion ». Le modèle pré-entraîné que nous utilisons, qui a été formé sur un corpus plus large de phrases en français et en anglais, choisit souvent l'option plus simple en laissant le mot tel quel.

In [5]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"
translator = pipeline("translation", model=model_checkpoint)

c:\Users\Zakaria-Laptop\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
c:\Users\Zakaria-Laptop\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
translator("Default to expanded threads")

[{'translation_text': 'Par défaut pour les threads élargis'}]

Un autre exemple de ce comportement est le mot « plugin », qui n'est pas officiellement un mot français mais que la plupart des francophones comprennent sans le traduire. Dans le jeu de données KDE4, ce mot a été traduit en français par l'expression plus officielle « module d'extension »

In [7]:
split_datasets["train"][172]["translation"]

{'en': 'Unable to import %1 using the OFX importer plugin. This file is not the correct format.',
 'fr': "Impossible d'importer %1 en utilisant le module d'extension d'importation OFX. Ce fichier n'a pas un format correct."}

In [8]:
translator(split_datasets["train"][172]["translation"]["en"])

[{'translation_text': "Impossible d'importer %1 en utilisant le plugin d'importateur OFX. Ce fichier n'est pas le bon format."}]

#### 2 - Processing the data

les textes doivent être convertis en ensembles d'ID de tokens pour que le modèle puisse les comprendre. Pour cette tâche, nous devons tokeniser à la fois les entrées et les cibles. Notre première étape est de créer l'objet tokenizer.

Comme mentionné précédemment, nous utiliserons un modèle Marian pré-entraîné pour la traduction de l'anglais vers le français. Si vous utilisez ce code avec une autre paire de langues, veillez à adapter le point de contrôle du modèle. L'organisation Helsinki-NLP propose plus d'un millier de modèles dans plusieurs langues.

In [9]:
checkpoint = "Helsinki-NLP/opus-mt-en-fr"

tokenizer = AutoTokenizer.from_pretrained(checkpoint, return_tensors = "pt")

La préparation de nos données est assez simple. Il y a juste une chose à retenir : il faut s'assurer que le tokenizer traite les cibles dans la langue de sortie (ici, le français). Vous pouvez le faire en passant les cibles à l'argument __text_targets__ de la méthode __call__ du tokenizer.

In [10]:
en_sentence = split_datasets["train"][120]["translation"]["en"]
fr_sentence = split_datasets["train"][120]["translation"]["fr"]

inputs = tokenizer(en_sentence, text_target = fr_sentence)
print(inputs)

{'input_ids': [12406, 4, 9432, 26, 29464, 746, 24, 1637, 212, 28, 479, 3, 443, 10042, 24, 63, 2959, 517, 28, 32, 15108, 2, 4, 9432, 32, 801, 26, 1265, 9929, 246, 3, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [40276, 34, 3404, 5, 2099, 27, 19, 14776, 24, 3432, 92, 167, 23, 15, 102, 350, 14, 6, 9313, 153, 402, 29033, 15080, 402, 29033, 416, 43, 806, 17598, 2, 19, 3404, 5, 2099, 81, 6, 82, 5644, 29, 27, 16, 1871, 20, 6, 28349, 3, 0]}


Comme nous pouvons le constater, la sortie contient les IDs d'entrée associés à la phrase en anglais, tandis que les IDs associés à la phrase en français sont stockés dans le champ des labels. Si vous oubliez de préciser que vous êtes en train de tokeniser des labels, ils seront traités par le tokenizer des entrées, ce qui, dans le cas d'un modèle Marian, ne fonctionnera pas du tout correctement.

In [11]:
wrong_targets = tokenizer(fr_sentence)
print(tokenizer.convert_ids_to_tokens(wrong_targets["input_ids"]))
print(tokenizer.convert_ids_to_tokens(inputs["labels"]))

['▁Sai', 's', 'isse', 'z', '▁un', '▁mo', 't', '▁de', '▁pass', 'e', '▁pour', '▁le', '▁dé', 'mar', 'rage', '▁(', 'si', '▁il', '▁y', '▁en', '▁a', ').', '▁Si', '▁l', "'", 'option', '▁«', '▁&', '▁#1', '60', ';', '▁rest', 're', 'int', '▁&', '▁#1', '60', ';', '▁»', '▁est', '▁co', 'ché', 'e', ',', '▁le', '▁mo', 't', '▁de', '▁pass', 'e', '▁n', "'", 'est', '▁re', 'qui', 's', '▁que', '▁pour', '▁les', '▁change', 'ments', '▁d', "'", 'option', 's', '.', '</s>']
['▁Saisissez', '▁un', '▁mot', '▁de', '▁passe', '▁pour', '▁le', '▁démarrage', '▁(', 'si', '▁il', '▁y', '▁en', '▁a', ').', '▁Si', '▁l', "'", 'option', '▁«', '▁&', '▁#160;', '▁restreint', '▁&', '▁#160;', '▁»', '▁est', '▁co', 'chée', ',', '▁le', '▁mot', '▁de', '▁passe', '▁n', "'", 'est', '▁requis', '▁que', '▁pour', '▁les', '▁changements', '▁d', "'", 'options', '.', '</s>']


Comme nous pouvons le voir, utiliser le tokenizer anglais pour prétraiter une phrase en français entraîne un nombre beaucoup plus élevé de tokens, car le tokenizer ne connaît pas les mots français (sauf ceux qui apparaissent également en anglais, comme « discussion »).

Puisque les entrées sont un dictionnaire contenant nos clés habituelles (IDs d'entrée, masque d'attention, etc.), la dernière étape consiste à définir la fonction de prétraitement que nous appliquerons aux jeux de données.

In [12]:
max_length = 128

def preprocess_function(exemple: str):
    """une fct pour tokenize nos textes d'entrée et de sortie

    Args:
        exemple (str): _description_

    Returns:
        _type_: _description_
    """    
    inputs = [ex["en"] for ex in exemple["translation"]]
    targets = [ex["fr"] for ex in exemple["translation"]]

    model_inputs = tokenizer(targets, text_target=targets, max_length=max_length, truncation=True)
    return model_inputs

In [13]:
tokenized_datasets = split_datasets.map(preprocess_function, batched=True, 
                                        remove_columns=split_datasets["train"].column_names,)

In [14]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 189155
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 21018
    })
})

#### 3 - Fine-tuning the model with the Trainer API

Le code réel utilisant le Trainer sera le même que précédemment, avec une petite différence : nous utilisons ici un $Seq2SeqTrainer$, qui est une sous-classe de Trainer permettant de gérer correctement l'évaluation, en utilisant la méthode __generate()__ pour prédire les sorties à partir des entrées. Nous examinerons cela plus en détail lorsque nous aborderons le calcul des métriques.

Tout d'abord, nous avons besoin d'un modèle réel à affiner. Nous utiliserons l'API AutoModel habituelle.

In [15]:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

##### Data Collation

Nous aurons besoin d'un data collator pour gérer le padding lors du dynamic batching. Nous ne pouvons pas simplement utiliser un __DataCollatorWithPadding__, car celui-ci ne fait le padding que pour les entrées (IDs d'entrée, masque d'attention, et types de tokens). Nos labels doivent également être complétés jusqu'à la longueur maximale rencontrée. Comme mentionné précédemment, la valeur de padding utilisée pour les labels doit être -100 et non le token de padding du tokenizer, afin que ces valeurs soient ignorées lors du calcul de la perte.

Tout cela est géré par un __DataCollatorForSeq2Seq__. Comme le __DataCollatorWithPadding__, il utilise le tokenizer pour prétraiter les entrées, mais il prend également le modèle. En effet, ce collator est responsable de préparer les IDs d'entrée du décodeur, qui sont des versions décalées des labels avec un token spécial au début. Étant donné que ce décalage varie selon les architectures, le __DataCollatorForSeq2Seq__ doit connaître l'objet modèle.

$ Explication$

__1. DataCollatorWithPadding :__

Usage principal : Généralement utilisé pour les tâches de classification, d'extraction d'information ou tout autre type de tâche où il est nécessaire de padder (remplir) les séquences pour qu'elles aient toutes la même longueur dans un batch.

Fonctionnement : Il ajuste la longueur des séquences de manière dynamique au sein d'un batch, en ajoutant des tokens de padding ([PAD]) pour rendre chaque séquence de la même longueur, sans toucher aux labels. Cela est particulièrement utile pour les modèles de type BERT, RoBERTa, etc., où les séquences d'entrée peuvent avoir des longueurs variables.

Exemple :

Séquences originales : [[1, 2, 3], [1, 2], [1]]
Après padding : [[1, 2, 3], [1, 2, 0], [1, 0, 0]]

__2. DataCollatorForSeq2Seq :__

Usage principal : Spécifiquement conçu pour les tâches de génération de séquences comme la traduction ou le résumé de texte, où l'on traite des modèles Seq2Seq tels que T5, BART, etc.

Fonctionnement : En plus de gérer le padding comme DataCollatorWithPadding, il gère également les labels (les séquences cibles). Les labels doivent aussi être paddés dans les tâches Seq2Seq pour que toutes les séquences cibles aient la même longueur dans un batch. De plus, il gère des aspects spécifiques comme l'ignoration des tokens de padding dans les labels pour ne pas pénaliser le modèle lorsqu'il prédit ces tokens lors de l'entraînement.

Exemple :

Séquences d'entrée : [[1, 2, 3], [1, 2]]
Labels (séquences cibles) : [[4, 5, 6], [4, 5]]
Après padding (entrée et labels) :
Séquences d'entrée : [[1, 2, 3], [1, 2, 0]]
Labels : [[4, 5, 6], [4, 5, -100]] (où -100 indique que le padding ne sera pas pris en compte dans la perte)

In [17]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [20]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])
batch.keys

<bound method BatchEncoding.keys of {'input_ids': tensor([[  577,  1205,   483, 13077,     2,  1205,   517, 13926,  1588,    16,
          3842,     9,     5,  1710,     0, 59513, 59513],
        [ 1211,     3,    49,  9409,  1211,     3, 29140,   817,  3124,   817,
          4274,  3534,   794,  7907, 24842,  3386,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([[  577,  5891,     2,  3184,    16,  2542,     5,  1710,     0,  -100,
          -100,  -100,  -100,  -100,  -100,  -100],
        [ 1211,     3,    49,  9409,  1211,     3, 29140,   817,  3124,   817,
           550,  7032,  5821,  7907, 12649,     0]]), 'decoder_input_ids': tensor([[59513,   577,  5891,     2,  3184,    16,  2542,     5,  1710,     0,
         59513, 59513, 59513, 59513, 59513, 59513],
        [59513,  1211,     3,    49,  9409,  1211,     3, 29140,   817,  3124,
           817,   550,  7

In [25]:
batch["labels"]

tensor([[  577,  5891,     2,  3184,    16,  2542,     5,  1710,     0,  -100,
          -100,  -100,  -100,  -100,  -100,  -100],
        [ 1211,     3,    49,  9409,  1211,     3, 29140,   817,  3124,   817,
           550,  7032,  5821,  7907, 12649,     0]])

Nous pouvons également examiner les IDs d'entrée du décodeur pour vérifier qu'ils sont des versions décalées des labels.

In [32]:
batch["decoder_input_ids"]

tensor([[59513,   577,  5891,     2,  3184,    16,  2542,     5,  1710,     0,
         59513, 59513, 59513, 59513, 59513, 59513],
        [59513,  1211,     3,    49,  9409,  1211,     3, 29140,   817,  3124,
           817,   550,  7032,  5821,  7907, 12649]])

Nous allons passer ce __data_collator__ au __Seq2SeqTrainer__. Ensuite, examinons la métrique.

Voici la traduction et le résumé :

La métrique traditionnelle utilisée pour la traduction est le score BLEU, introduit dans un article de 2002 par Kishore Papineni et al. Le score BLEU évalue à quel point les traductions sont proches de leurs labels. Il ne mesure pas l'intelligibilité ou la correction grammaticale des sorties générées par le modèle, mais applique des règles statistiques pour vérifier que tous les mots des sorties générées apparaissent également dans les cibles. De plus, des règles pénalisent les répétitions de mots si elles ne sont pas présentes dans les cibles (afin d'éviter que le modèle génère des phrases comme « the the the ») et les phrases trop courtes (comme « the »).

Une faiblesse du score BLEU est qu'il nécessite que le texte soit déjà tokenisé, ce qui complique la comparaison entre des modèles utilisant différents tokenizers. C'est pourquoi la métrique la plus couramment utilisée aujourd'hui pour évaluer les modèles de traduction est SacreBLEU, qui corrige ce défaut (et d'autres) en standardisant l'étape de tokenisation. Pour utiliser cette métrique, nous devons d'abord installer la bibliothèque SacreBLEU.

In [33]:
!pip install sacrebleu

Using cached tabulate-0.9.0-py3-none-any.whl (35 kB)
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ----- ---------------------------------- 0.5/3.8 MB 3.4 MB/s eta 0:00:01
   ------------- -------------------------- 1.3/3.8 MB 3.4 MB/s eta 0:00:01
   ------------- -------------------------- 1.3/3.8 MB 3.4 MB/s eta 0:00:01
   ------------- -------------------------- 1.3/3.8 MB 3.4 MB/s eta 0:00:01
   ------------- -------------------------- 1.3/3.8 MB 3.4 MB/s eta 0:00:01
   ------------- -------------------------- 1.3/3.8 MB 3.4 MB/s eta 0:00:01
   ---------------- ----------------------- 1.6/3.8 MB 1.1 MB/s eta 0:00:03
   ---------------- ----------------------- 1.6/3.8 MB 1.1 MB/s eta 0:00:03
   ---------------- ----------------------- 1.6/3.8 MB 1.1 MB/s eta 0:00:03
   ---------------- ----------------------- 1.6/3.8 MB 1.1 MB/s eta 0:00:03
   ------------------- -------------------- 1.8/3.8 MB 740.2 kB/s eta 0:00:03
   --------------------- ---------------

In [37]:
import evaluate

metric = evaluate.load("sacrebleu")

Cette métrique prend des textes en tant qu'entrées et cibles. Elle est conçue pour accepter plusieurs cibles possibles, car il existe souvent plusieurs traductions acceptables d'une même phrase. Le jeu de données que nous utilisons n'en fournit qu'une, mais dans le domaine du traitement du langage naturel (NLP), il n'est pas rare de trouver des jeux de données avec plusieurs phrases en tant que labels. Ainsi, les prédictions doivent être une liste de phrases, tandis que les références doivent être une liste de listes de phrases.

Essayons un exemple.

In [38]:
predictions = [
    "This plugin lets you translate web pages between several languages automatically."
]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 46.750469682990186,
 'counts': [11, 6, 4, 3],
 'totals': [12, 11, 10, 9],
 'precisions': [91.66666666666667,
  54.54545454545455,
  40.0,
  33.333333333333336],
 'bp': 0.9200444146293233,
 'sys_len': 12,
 'ref_len': 13}

In [39]:
predictions = ["This This This This"]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 1.683602693167689,
 'counts': [1, 0, 0, 0],
 'totals': [4, 3, 2, 1],
 'precisions': [25.0, 16.666666666666668, 12.5, 12.5],
 'bp': 0.10539922456186433,
 'sys_len': 4,
 'ref_len': 13}

In [40]:
predictions = ["This plugin"]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 0.0,
 'counts': [2, 1, 0, 0],
 'totals': [2, 1, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'bp': 0.004086771438464067,
 'sys_len': 2,
 'ref_len': 13}

Pour passer des sorties du modèle aux textes utilisables par la métrique, nous allons utiliser la méthode __tokenizer.batch_decode()__. Il nous suffit de supprimer tous les -100 dans les labels (le tokenizer s'occupera automatiquement de faire de même pour le token de padding).